## Step 1: Create table and load demo data

In [ ]:

-- Drop and recreate
DROP TABLE ORDERITEM PURGE;

CREATE TABLE ORDERITEM (
  ORDERITEM_ID NUMBER(10) PRIMARY KEY,
  ORDER_DATE   DATE,              -- stores date + time
  PRODUCTNAME  VARCHAR2(50),
  SALES        NUMBER(10)
);

INSERT INTO ORDERITEM VALUES (1,  DATE '2025-01-05', 'Apple iPhone', 50000);
INSERT INTO ORDERITEM VALUES (2,  DATE '2025-01-08', 'Google Pixel', 30000);
INSERT INTO ORDERITEM VALUES (3,  DATE '2025-01-15', 'Samsung S25', 40000);
INSERT INTO ORDERITEM VALUES (4,  DATE '2025-01-20', 'Apple iPhone', 52000);
INSERT INTO ORDERITEM VALUES (5,  DATE '2025-01-25', 'Google Pixel', 31000);

INSERT INTO ORDERITEM VALUES (6,  DATE '2025-02-07', 'Apple iPhone', 55000);
INSERT INTO ORDERITEM VALUES (7,  DATE '2025-02-07', 'Samsung S25', 48000);
INSERT INTO ORDERITEM VALUES (8,  DATE '2025-02-07', 'Google Pixel', 36000);
INSERT INTO ORDERITEM VALUES (9,  DATE '2025-02-07', 'Google Pixel', 35000);
INSERT INTO ORDERITEM VALUES (10, DATE '2025-02-12', 'Samsung S25', 45000);
INSERT INTO ORDERITEM VALUES (11, DATE '2025-02-20', 'Apple iPhone', 57000);
INSERT INTO ORDERITEM VALUES (12, DATE '2025-02-27', 'Google Pixel', 36000);
INSERT INTO ORDERITEM VALUES (13, DATE '2025-02-28', 'Samsung S25', 47000);

COMMIT;


## Step 2: Explore the data

In [ ]:

SELECT ORDERITEM_ID, ORDER_DATE, PRODUCTNAME, SALES
FROM ORDERITEM
ORDER BY ORDER_DATE, ORDERITEM_ID;


## Step 3: Total sales (simple aggregate)

In [ ]:
SELECT SUM(SALES) AS TOTAL_SALES FROM ORDERITEM;

## Step 4: GROUP BY pitfalls (these illustrate common mistakes)

In [ ]:

-- ORA-00937
SELECT ORDER_DATE, PRODUCTNAME, SUM(SALES) AS TOTAL_SALES
FROM ORDERITEM;


In [ ]:

-- ORA-00979
SELECT ORDER_DATE, PRODUCTNAME, SALES, SUM(SALES) AS TOTAL_SALES
FROM ORDERITEM
GROUP BY ORDER_DATE;


In [ ]:

-- Syntactically valid but not insightful
SELECT ORDER_DATE, PRODUCTNAME, SALES, SUM(SALES) AS TOTAL_SALES
FROM ORDERITEM
GROUP BY ORDER_DATE, PRODUCTNAME, SALES
ORDER BY ORDER_DATE, ORDERITEM_ID;


## Step 5: Analytic grand total per row (`OVER()`)

In [ ]:

SELECT ORDER_DATE, PRODUCTNAME, SALES,
       SUM(SALES) OVER() AS TOTAL_SALES
FROM ORDERITEM
ORDER BY ORDER_DATE, ORDERITEM_ID;


## Step 6: Monthly total (partition by month)
Use `TRUNC(ORDER_DATE,'MM')` to group by calendar month.

In [ ]:

SELECT TRUNC(ORDER_DATE,'MM') AS ORDER_MONTH, PRODUCTNAME, SALES,
       SUM(SALES) OVER (PARTITION BY TRUNC(ORDER_DATE,'MM')) AS MONTHLY_TOTAL
FROM ORDERITEM
ORDER BY ORDER_MONTH, ORDERITEM_ID;


## Step 7: Running monthly total (within each month)

In [ ]:

SELECT TRUNC(ORDER_DATE,'MM') AS ORDER_MONTH, PRODUCTNAME, SALES,
       SUM(SALES) OVER (
         PARTITION BY TRUNC(ORDER_DATE,'MM')
         ORDER BY ORDER_DATE, ORDERITEM_ID
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS RUNNING_MONTHLY_TOTAL
FROM ORDERITEM
ORDER BY ORDER_MONTH, ORDERITEM_ID;
